In [17]:
! pip install -r requirements.txt   

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [19]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [20]:
data.drop(['RowNumber'	,'CustomerId',	'Surname'],axis=1,inplace=True)

In [21]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [22]:
onehot_encoder_geo=OneHotEncoder()
geo_encoded=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [23]:
data=pd.concat([data.drop(['Geography'],axis=1),geo_encoded_df],axis=1)

In [24]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [25]:
x=data.drop(['EstimatedSalary'],axis=1)
y=data['EstimatedSalary']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [26]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [27]:
with open('scaler.pkl','wb') as f:
  pickle.dump(scaler,f)

with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender,f)

with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(onehot_encoder_geo,f)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime
 


In [29]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)## Default is linear
])
opt=tf.keras.optimizers.Adam(learning_rate=0.001)
loss=tf.keras.losses.mean_absolute_error
model.compile(optimizer=opt,loss=loss,metrics=['mae'])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
x.columns

Index(['CreditScore', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts',
       'HasCrCard', 'IsActiveMember', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain'],
      dtype='object')

In [35]:
y

0       101348.88
1       112542.58
2       113931.57
3        93826.63
4        79084.10
          ...    
9995     96270.64
9996    101699.77
9997     42085.58
9998     92888.52
9999     38190.78
Name: EstimatedSalary, Length: 10000, dtype: float64

In [31]:
log_dir='logs/fit'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [32]:
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)
early_stopping_callbacks=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
history=model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callbacks])
model.save('model.h5')
%load_ext tensorboard
%tensorboard --logdir logs/fit

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 100087.1875 - mae: 100087.1875 - val_loss: 99657.0547 - val_mae: 99657.0547
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99306.9453 - mae: 99306.9453 - val_loss: 98101.3906 - val_mae: 98101.3906
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 96589.7031 - mae: 96589.7031 - val_loss: 94167.4375 - val_mae: 94167.4375
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 91267.7266 - mae: 91267.7266 - val_loss: 87581.9141 - val_mae: 87581.9141
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 83609.1953 - mae: 83609.1953 - val_loss: 79022.6875 - val_mae: 79022.6875
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 74636.5938 - mae: 74636.5938 - val_loss: 69933.2812 - val_mae: 69933.2812
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 65867.4453 - mae: 65867.4453 

C:\Users\KIIT0001\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Reusing TensorBoard on port 6006 (pid 17288), started 0:14:13 ago. (Use '!kill 17288' to kill it.)